<a href="https://colab.research.google.com/github/Shopping-Yuan/ML_Project/blob/Shopping_vscode_branch/GBDT/GBDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'house-prices-advanced-regression-techniques:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F5407%2F868283%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240531%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240531T055519Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7deedf605096331826d7d615f732af1b6b4aaf973f2d442d98476731535a687b83bab3ee9175a6c5987fc029fcb683f2bb8765a1d102fe6d136bf105e29aaa1dd01c303765f1760f0e3794ea7a66aaded435c7437603ce0e8a0378e1f277eb3daf82229bd9a8be6fc32e22a756c173c07fc20afed16a55befdfdcca9ceb1b81842c2d69d9b8f745606baaa255da55f896b8f0e653c72bbc1d0b6195b6e6337e326bb0eb2e0dc645fa6a8ab8d0edbcb3790c7163af53c795d172bd50e3717950b3417266db705052e79a46abf92d3861dab65f06156ac1a0be0a4b763699440e54eee8c98f62cac71e8cc89915374abca343c951a5127d36cce49880f253f435f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 203809 bytes downloaded
Downloaded and uncompressed: house-prices-advanced-regression-techniques
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt


In [16]:
from sklearn.model_selection import train_test_split
train_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [41]:
train_df.dtypes
null = train_df.isnull().sum()
notnull = train_df.notnull().sum()
miss_portion = null[null > 0]/(null[null > 0]+notnull[null > 0])
drop = miss_portion[miss_portion > 0.5].index.values
drop + [""]

array(['Alley', 'MasVnrType', 'PoolQC', 'Fence', 'MiscFeature'],
      dtype=object)

In [ ]:
# from sklearn.preprocessing import LabelEncoder
def labelencoder(dataset):
    names = list(train.dtypes[train.dtypes == object].index)
    dataset_encoder = pd.DataFrame()
    for name in dataset.columns:
        if name in names:
            # label_encoder = LabelEncoder()
            # dataset_encoder[name] = label_encoder.fit_transform(dataset[name])
            dataset_encoder = pd.concat([dataset_encoder,pd.get_dummies(dataset[name])],axis = 1)
        else :
            dataset_encoder[name] = dataset[name]
    return dataset_encoder


train_encoder = labelencoder(train.drop(["Id"],axis = 1))
test_encoder = labelencoder(test.drop(["Id"],axis = 1))

In [ ]:
train_encoder.head(20)
len(train_encoder.columns)
train_encoder.isnull().any()
# train_encoder.shape

MSSubClass    False
C (all)       False
FV            False
RH            False
RL            False
              ...  
Alloca        False
Family        False
Normal        False
Partial       False
SalePrice     False
Length: 288, dtype: bool

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
impute_training_set = pd.DataFrame(imputer.fit_transform(train_encoder),columns = list(train_encoder.columns))
impute_testing_set = pd.DataFrame(imputer.fit_transform(test_encoder),columns = list(test_encoder.columns))
impute_training_set.head(20)

,MSSubClass,C (all),FV,RH,RL,RM,LotFrontage,LotArea,Grvl,Pave,...,New,Oth,WD,Abnorml,AdjLand,Alloca,Family,Normal,Partial,SalePrice
0,60.0,0.0,0.0,0.0,1.0,0.0,65.0,8450.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,208500.0
1,20.0,0.0,0.0,0.0,1.0,0.0,80.0,9600.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,181500.0
2,60.0,0.0,0.0,0.0,1.0,0.0,68.0,11250.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,223500.0
3,70.0,0.0,0.0,0.0,1.0,0.0,60.0,9550.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,140000.0
4,60.0,0.0,0.0,0.0,1.0,0.0,84.0,14260.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,250000.0
5,50.0,0.0,0.0,0.0,1.0,0.0,85.0,14115.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,143000.0
6,20.0,0.0,0.0,0.0,1.0,0.0,75.0,10084.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,307000.0
7,60.0,0.0,0.0,0.0,1.0,0.0,78.0,10382.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,200000.0
8,50.0,0.0,0.0,0.0,0.0,1.0,51.0,6120.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,129900.0
9,190.0,0.0,0.0,0.0,1.0,0.0,50.0,7420.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,118000.0


In [ ]:
train_set,val_set = train_test_split(impute_training_set, test_size=0.2, random_state=1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(train_set.drop(["SalePrice"],axis = 1), train_set["SalePrice"])
# clf.score(X_test, y_test)

In [ ]:
clf.score(val_set.drop(["SalePrice"],axis = 1), val_set["SalePrice"])

0.003424657534246575

In [ ]:
clf.predict(impute_testing_set)


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- 2.5Fin
- ClyTile
- Floor
- Membran
- Metal
- ...
